# New ML Active Learning Workflow
---

# Import Modules

In [1]:
import os
import sys

sys.path.insert(0, os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/ml_convergence_plots"))

import pickle
import random
import copy

import pandas as pd
import numpy as np
import plotly.plotly as py

from ase.visualize import view
from ase import io

# #############################################################################
# #############################################################################
from catlearn.regression import GaussianProcess
import gpflow

# #############################################################################
# #############################################################################
from methods import process_data_for_plot, get_layout

# Script Inputs

# Read Data

In [4]:
path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/parsing_chris_dft_data",
    "df_dft_calcs.pickle")
with open(path_i, "rb") as fle:
    df_dft_calcs = pickle.load(fle)

# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling",
    "df_oqmd_data.pickle")
with open(path_i, "rb") as fle:
    df_oqmd_data = pickle.load(fle)

df_train = pd.concat([df_oqmd_data, df_dft_calcs], sort=False)
# Removing missing data
df_train = df_train[df_train["atoms"].notnull()]
df_train = df_train.drop(["energy", "force_max", "force_sum"], axis=1)

# #############################################################################
# #############################################################################
# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "chris_prototypes_structures",
    "data_structures.pickle")
with open(path_i, "rb") as fle:
    df_struct = pickle.load(fle)

# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "chris_prototypes_structures",
    "data_prototypes.pickle")
with open(path_i, "rb") as fle:
    df_proto = pickle.load(fle)

# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "data/ml_irox_data",
    "unique_ids.csv")
df_ids = pd.read_csv(path_i)

# #############################################################################

path_i = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/ml_modelling/00_ml_workflow/190611_new_workflow",
    "data",
    "df_features_pca.pickle",
    )
with open(path_i, "rb") as fle:
    df_features_pca = pickle.load(fle)

# Finding Structure that is closest to beta-IrO3 phase

In [9]:
# AB3_8_ik_glm2_66
# npvumg7g93

df_proto[df_proto["spacegroup_i"] == 66]

,name_i,parameter_values_i,spacegroup_i,species_i,wyckoff_i
id,,,,,
7g8hvsnlbt,AB2_10_am_hklm_66,"[{'name': 'a', 'value': 12.5345}, {'name': 'b/...",66,"[Ir, Ir, O, O, O, O]","[a, m, h, k, l, m]"
npvumg7g93,AB3_8_acl_klm2_66,"[{'name': 'a', 'value': 9.24458}, {'name': 'b/...",66,"[Ir, Ir, Ir, O, O, O, O]","[a, c, l, k, l, m, m]"


# Finding Structure that is closest to alpha-IrO3 phase

In [19]:
# AB3_2_b_e_167
# 8k7expx2bp

df_proto[df_proto["spacegroup_i"] == 167]


row_i = df_struct.loc["8k7expx2bp"]
view(row_i["atoms"])

# view(df_struct.loc["8k7expx2bp"]["atoms"])

In [20]:
row_i

atoms     (Atom('O', [2.0795108014000006, 3.601818362913...
id_old                                                  231
path      04_irox_surfaces_oer/chris_prototypes_structur...
source                                                chris
stoich                                                  AB3
Name: 8k7expx2bp, dtype: object

# Getting minimum energy line trace in animation